In [ ]:
#Stworzenie podziału na dane uczące i walidacyjne
import os, shutil, random, glob
from pathlib import Path

src_img = Path("D:/AAA/Nowy folder/Outputs/project-8-at-2025-08-12-11-44-492dcff8/images")
src_lab = Path("D:/AAA/Nowy folder/Outputs/project-8-at-2025-08-12-11-44-492dcff8/labels")
out = Path("D:/AAA/Nowy folder/Outputs/project-8-at-2025-08-12-11-44-492dcff8/yolo_pupil_seg"); random.seed(42)
for p in ['images/train','images/val','labels/train','labels/val']:
    (out/p).mkdir(parents=True, exist_ok=True)

imgs = [Path(p) for p in glob.glob(str(src_img/'*'))]
random.shuffle(imgs); n = int(0.8*len(imgs))
splits = {'train': imgs[:n], 'val': imgs[n:]}

for split, files in splits.items():
    for img in files:
        lbl = src_lab/(img.stem + ".txt")
        shutil.copy2(img, out/'images'/split/img.name)
        if lbl.exists(): shutil.copy2(lbl, out/'labels'/split/lbl.name)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-seg.pt')  # lub yolov8n-seg.pt
model.train(data='D:/AAA/Nowy folder/Outputs/project-8-at-2025-08-12-11-44-492dcff8/data.yaml', imgsz=400, epochs=100, batch=16, device=0, workers=8, project='runs', name='pupil-seg')

Ultralytics 8.3.177  Python-3.11.9 torch-2.8.0+cpu CPU (Intel Core(TM) i7-8750H 2.20GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/AAA/Nowy folder/Outputs/project-8-at-2025-08-12-11-44-492dcff8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pupil-seg4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n-seg summary: 151 layers, 3,263,811 parameters, 3,263,795 gradients, 12.1 GFLOPs

Transferred 381/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access  (ping: 0.10.0 ms, read: 12.41.2 MB/s, size: 78.4 KB)


train: Scanning D:\AAA\Nowy folder\Outputs\project-8-at-2025-08-12-11-44-492dcff8\yolo_pupil_seg\labels\train... 29 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29/29 [00:00<00:00, 293.57it/s]

train: New cache created: D:\AAA\Nowy folder\Outputs\project-8-at-2025-08-12-11-44-492dcff8\yolo_pupil_seg\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 12.71.5 MB/s, size: 80.7 KB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


                   all          8          8      0.994          1      0.995      0.935      0.994          1      0.995      0.935
Speed: 1.6ms preprocess, 107.0ms inference, 0.0ms loss, 32.8ms postprocess per image
Results saved to runs\pupil-seg4


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020C10464F10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO(r"D:\AAA\Nowy folder\pupiltracking\runs\pupil-seg4\weights\best.pt")
results = model.predict(
    source=r"D:\AAA\Nowy folder\Outputs\Test",  # może być plik lub folder
    conf=0.35, max_det=1, device=0, save=True, imgsz=320
)


image 1/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.25.48.png: 320x320 1 Pupil, 10.7ms
image 2/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.08.png: 320x320 1 Pupil, 10.7ms
image 3/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.18.png: 320x320 1 Pupil, 11.1ms
image 4/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.22.png: 320x320 1 Pupil, 10.7ms
image 5/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.48.png: 320x320 1 Pupil, 10.9ms
image 6/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.42.36.png: 320x320 1 Pupil, 11.0ms
image 7/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.24.png: 320x320 1 Pupil, 10.8ms
image 8/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.46.png: 320x320 1 Pupil, 10.6ms
image 9/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.50.png: 320x320 1 Pupil, 10.5ms
image 10/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.54.png: 320x320 1 Pupil, 11.3ms
image 11/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.58.png: 320x320 1

In [9]:
from ultralytics import YOLO
import cv2
import os
import glob
import numpy as np

# Ścieżka wejściowa
input_dir = r"D:\AAA\Nowy folder\Outputs\Test"

# Wczytanie modelu segmentacyjnego YOLO
model = YOLO(r"D:\AAA\Nowy folder\pupiltracking\runs\pupil-seg4\weights\best.pt")

# Pobranie plików graficznych
image_files = glob.glob(os.path.join(input_dir, "*.jpg")) + \
              glob.glob(os.path.join(input_dir, "*.png")) + \
              glob.glob(os.path.join(input_dir, "*.jpeg"))

for img_path in image_files:
    img = cv2.imread(img_path)

    # Detekcja
    results = model.predict(
        source=img_path,
        conf=0.35, max_det=1, device=0, save=False, imgsz=320
    )

    for r in results:
        if r.masks is not None:
            masks = r.masks.xy  # lista poligonów (każdy to tablica Nx2)
            for poly in masks:
                # Konwersja do int32 dla OpenCV
                pts = np.array(poly, dtype=np.int32)
                pts = pts.reshape((-1, 1, 2))

                # Rysowanie poligonu
                cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

                # Obliczanie środka poligonu (momenty geometryczne)
                M = cv2.moments(pts)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                else:
                    cx, cy = 0, 0

                # Rysowanie środka
                cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(img, f"({cx},{cy})", (cx + 10, cy - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Wyświetlanie obrazu
    cv2.imshow("Poligon + środek", img)
    key = cv2.waitKey(0)
    if key == 27:  # ESC → przerwij
        break

cv2.destroyAllWindows()



image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.25.48.png: 320x320 1 Pupil, 13.7ms
Speed: 1.2ms preprocess, 13.7ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.08.png: 320x320 1 Pupil, 27.2ms
Speed: 1.8ms preprocess, 27.2ms inference, 4.8ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.18.png: 320x320 1 Pupil, 24.7ms
Speed: 1.7ms preprocess, 24.7ms inference, 7.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.22.png: 320x320 1 Pupil, 22.6ms
Speed: 1.5ms preprocess, 22.6ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.48.png: 320x320 1 Pupil, 55.2ms
Speed: 2.1ms preprocess, 55.2ms inference, 5.5ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-1

In [14]:
from ultralytics import YOLO
import cv2
import os
import glob
import numpy as np

# Ścieżka wejściowa
input_dir = r"D:\\AAA\\Nowy folder\\Outputs\\Test"

# Wczytanie modelu segmentacyjnego YOLO
model = YOLO(r"D:\\AAA\\Nowy folder\\pupiltracking\\runs\\pupil-seg4\\weights\\best.pt")

# Pobranie plików graficznych
image_files = glob.glob(os.path.join(input_dir, "*.jpg")) + \
              glob.glob(os.path.join(input_dir, "*.png")) + \
              glob.glob(os.path.join(input_dir, "*.jpeg"))

for img_path in image_files:
    img = cv2.imread(img_path)

    # Detekcja
    results = model.predict(
        source=img_path,
        conf=0.35, max_det=1, device=0, save=False, imgsz=320
    )

    for r in results:
        if r.masks is not None:
            masks = r.masks.xy
            for poly in masks:
                pts = np.array(poly, dtype=np.int32)
                pts_reshaped = pts.reshape((-1, 1, 2))

                # Rysowanie poligonu
                cv2.polylines(img, [pts_reshaped], isClosed=True, color=(0, 255, 0), thickness=2)

                # Środek geometryczny
                M = cv2.moments(pts)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                else:
                    cx, cy = 0, 0

                # Znalezienie punktów przecięcia poziomej linii y=cy z krawędziami poligonu
                intersections = []
                for i in range(len(pts)):
                    x1, y1 = pts[i]
                    x2, y2 = pts[(i + 1) % len(pts)]  # następny punkt (zamknięty poligon)

                    # Sprawdzamy, czy odcinek przecina linię y=cy
                    if (y1 <= cy <= y2) or (y2 <= cy <= y1):
                        if y1 != y2:  # unikamy dzielenia przez zero
                            t = (cy - y1) / (y2 - y1)
                            x_int = int(x1 + t * (x2 - x1))
                            intersections.append((x_int, cy))

                if len(intersections) >= 2:
                    # Wybieramy najbardziej skrajne punkty
                    p_left = min(intersections, key=lambda p: p[0])
                    p_right = max(intersections, key=lambda p: p[0])
                    diameter_x = abs(p_right[0] - p_left[0])

                    # Rysowanie linii średnicy przez środek
                    cv2.line(img, p_left, p_right, (255, 0, 0), 2)
                    cv2.putText(img, f"{diameter_x}px",
                                ((p_left[0] + p_right[0]) // 2, cy + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                # Rysowanie środka
                cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(img, f"({cx},{cy})", (cx + 10, cy - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Wyświetlanie
    cv2.imshow("Poligon + środek + średnica przez środek", img)
    key = cv2.waitKey(0)
    if key == 27:
        break

cv2.destroyAllWindows()



image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.25.48.png: 320x320 1 Pupil, 10.6ms
Speed: 1.2ms preprocess, 10.6ms inference, 3.7ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.08.png: 320x320 1 Pupil, 14.8ms
Speed: 1.6ms preprocess, 14.8ms inference, 3.7ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.18.png: 320x320 1 Pupil, 21.1ms
Speed: 1.5ms preprocess, 21.1ms inference, 5.2ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.22.png: 320x320 1 Pupil, 23.4ms
Speed: 2.1ms preprocess, 23.4ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.48.png: 320x320 1 Pupil, 54.0ms
Speed: 1.9ms preprocess, 54.0ms inference, 5.7ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-1

KeyboardInterrupt: 

In [1]:
import torch
print(torch.cuda.is_available())  # powinno być True
print(torch.cuda.get_device_name(0))  # powinno pokazać GTX 1050 Ti


True
NVIDIA GeForce GTX 1050 Ti
